In [1]:
import polars as pl
df = pl.read_parquet('../../parquets/batting.parquet')

In [2]:
df = df.with_columns(
    pl.col("date")
    .cast(pl.String)
    .str.strptime(pl.Date, "%Y%m%d")
    .dt.year()
    .alias("year")
)

df.filter(~pl.col("gametype").is_in(["exhibition", "allstar"]))

gid,id,team,b_lp,b_seq,stattype,b_pa,b_ab,b_r,b_h,b_d,b_t,b_hr,b_rbi,b_sh,b_sf,b_hbp,b_w,b_iw,b_k,b_sb,b_cs,b_gdp,b_xi,b_roe,dh,ph,pr,date,number,site,vishome,opp,win,loss,tie,gametype,box,pbp,year
str,str,str,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,str,str,i64,i64,i64,str,str,str,i32
"""PHI190104180""","""davil101""","""BRO""",1,"""1""","""value""",5,5,1,1,0,0,0,0,0,null,0,0,null,0,0,null,null,0,null,null,null,null,19010418,null,"""PHI09""","""v""","""PHI""",1,0,0,"""regular""","""y""",null,1901
"""PHI190104180""","""keelw101""","""BRO""",2,"""1""","""value""",5,5,1,2,2,0,0,1,0,null,0,0,null,0,0,null,null,0,null,null,null,null,19010418,null,"""PHI09""","""v""","""PHI""",1,0,0,"""regular""","""y""",null,1901
"""PHI190104180""","""shecj101""","""BRO""",3,"""1""","""value""",5,4,4,3,0,3,0,1,0,null,0,1,null,0,0,null,null,0,null,null,null,null,19010418,null,"""PHI09""","""v""","""PHI""",1,0,0,"""regular""","""y""",null,1901
"""PHI190104180""","""kellj106""","""BRO""",4,"""1""","""value""",5,5,1,3,1,0,0,4,0,null,0,0,null,0,1,null,null,0,null,null,null,null,19010418,null,"""PHI09""","""v""","""PHI""",1,0,0,"""regular""","""y""",null,1901
"""PHI190104180""","""mccrt101""","""BRO""",5,"""1""","""value""",5,4,1,0,0,0,0,0,0,null,0,1,null,2,0,null,null,0,null,null,null,null,19010418,null,"""PHI09""","""v""","""PHI""",1,0,0,"""regular""","""y""",null,1901
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""NYA202410300""","""coleg001""","""NYA""",null,null,"""value""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null,null,null,20241030,0,"""NYC21""","""h""","""LAN""",0,1,0,"""worldseries""","""y""","""y""",2024
"""NYA202410300""","""holmc001""","""NYA""",null,null,"""value""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null,null,null,20241030,0,"""NYC21""","""h""","""LAN""",0,1,0,"""worldseries""","""y""","""y""",2024
"""NYA202410300""","""kahnt001""","""NYA""",null,null,"""value""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null,null,null,20241030,0,"""NYC21""","""h""","""LAN""",0,1,0,"""worldseries""","""y""","""y""",2024


In [3]:
cols_to_aggregate = [
    "b_pa", "b_ab", "b_r", "b_h", "b_d", "b_t", "b_hr", "b_rbi",
    "b_sh", "b_sf", "b_hbp", "b_w", "b_iw", "b_k", "b_sb", "b_cs",
    "b_gdp", "b_xi", "b_roe"
]
aggregations = [pl.sum(col).alias(col) for col in cols_to_aggregate]
yearly_df = (df
    .group_by(["id", "year"])
    .agg(aggregations)
    .sort(["year"])
)

In [5]:
yearly_adv_df = yearly_df.with_columns([
    (pl.col("b_h") / pl.col("b_ab")).alias("b_ba"),  # Batting Average
    ((pl.col("b_h") + pl.col("b_w") + pl.col("b_hbp")) / 
     (pl.col("b_ab") + pl.col("b_w") + pl.col("b_hbp") + pl.col("b_sf"))).alias("b_obp"),  # On-base Percentage
    ((pl.col("b_h") - pl.col("b_d") - pl.col("b_t") - pl.col("b_hr") + 
      2 * pl.col("b_d") + 3 * pl.col("b_t") + 4 * pl.col("b_hr")) / 
     pl.col("b_ab")).alias("b_slg"),  # Slugging Percentage
    (pl.col("b_k") / pl.col("b_ab")).alias("b_k_pct")  # Strikeout Rate
])

In [ ]:
yearly_adv_df = yearly_adv_df.with_columns(
    [
        pl.col("b_obp").shift(1).over("id").alias("lag_b_obp"),
        pl.col("b_slg").shift(1).over("id").alias("lag_b_slg"),
        pl.col("b_k_pct").shift(1).over("id").alias("lag_b_k_pct")
    ]
)

In [34]:
yearly_adv_df = yearly_adv_df.filter(
    pl.col("lag_b_obp").is_not_null() & pl.col("lag_b_obp").is_not_nan() & ~pl.col("lag_b_obp").is_infinite() &
    pl.col("lag_b_slg").is_not_null() & pl.col("lag_b_slg").is_not_nan() & ~pl.col("lag_b_slg").is_infinite() &
    pl.col("lag_b_k_pct").is_not_null() & pl.col("lag_b_k_pct").is_not_nan() & ~pl.col("lag_b_k_pct").is_infinite() &
    pl.col("b_ba").is_not_null() & pl.col("b_ba").is_not_nan() & ~pl.col("b_ba").is_infinite()
)

In [35]:
yearly_adv_df.write_parquet("../../parquets/batting-yearly-adv.parquet")

In [ ]:
training_data = yearly_adv_df.filter(pl.col("year") < 2020)

features = ["lag_b_obp", "lag_b_slg", "lag_b_k_pct"]

X = training_data.select(features).to_numpy()
y = training_data.select("b_ba").to_numpy().ravel()

In [ ]:
import xgboost as xgb

model = xgb.XGBRegressor(
    objective="reg:squarederror",
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    random_state=123
)

model.fit(X, y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.1, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=100,
             n_jobs=None, num_parallel_tree=None, ...)

In [43]:
model.save_model("../../models/xgboost.json")